# `SnB` Python API

## `ShakeNBreak` applied to Te interstitials in CdTe ($Te_{i}$)

In this notebook we show how to generate interstitial defects using `doped`, and apply `ShakeNBreak` (`SnB`) to them. To see the full `ShakeNBreak` workflow, have a look at the notebook `ShakeNBreak_Example_Workflow.ipynb` (recommended to view it in the [SnB Python API tutorial](https://shakenbreak.readthedocs.io/en/latest/ShakeNBreak_Example_Workflow.html) docs page).

## Table of contents
* [Generate defects with doped/pymatgen](#generate)
* [Apply SnB to defects](#SnB)
* [Send to HPCs and run calculations](#HPCs)

In [1]:
import os
import sys

import ase
import numpy as np
import pymatgen
from importlib_metadata import version

import shakenbreak

# Check versions
print("Pymatgen version:", version('pymatgen') )
print("Pymatgen-analysis-defects version:", version('pymatgen-analysis-defects') )
print("Ase version:", version('ase') )
print("ShakeNBreak version:", version('shakenbreak') )

Pymatgen version: 2023.2.22
Pymatgen-analysis-defects version: 2023.3.25
Ase version: 3.22.1
ShakeNBreak version: 23.2.8


#### Rationale for `SnB`   

Defect distortions often follow the change in electron count when introducing that defect to the system. For the neutral Cd vacancy ($V_{Cd}^0$) for example, the removal of Cd and its two valence electrons means that local distortions are likely to involve two neighbouring Te atoms moving closer/further apart to accommodate the broken bonds. For the singly-charged vacancy, we are likely to have just one neighbouring Te moving, etc. This isn't always the case, but typically points us in the right direction to search the PES, and has been confirmed to yield the best performance (see SI of [_Identifying the ground state structures of point defects in solids_](https://www.nature.com/articles/s41524-023-00973-1) Mosquera-Lois, Kavanagh, Walsh and Scanlon 2022).

So, the `SnB` method involves distorting the initial bond lengths around the defect for a mesh of trial distortions, with the number of neighbours to distort dictated by the change in valence electron count, performing coarse $\Gamma$-only (`vasp_gam`) relaxations and then comparing the final energies, to see if we identify any lower energy defect structures.

<a id='generate'></a>

## 1. Generate defects with `doped`

When using `doped` to generate your defects, this should be done in a different `python` virtual environment to the one with `ShakeNBreak`, as they currently require different `pymatgen` versions due to major changes in the defects code. The recommended installation of `doped` in a virtual environment is described on the [`doped README`](https://github.com/SMTG-UCL/doped/).

The easiest way to do this is to generate the defects in your `doped` environment (either in a separate notebook, or by restarting the `jupyter` server for the same notebook but in the `doped` environment), and then save these to a `json` file as shown below. Then, you can switch back to your `base`/`ShakeNBreak` environment and reload them to use with `ShakeNBreak` (shown later on below).

In [1]:
# To generate the Te interstitials with doped, we can use the code below.
# Described in more detail on the doped example: https://github.com/SMTG-UCL/doped/blob/master/examples/dope_workflow_example.ipynb
# As this requires pymatgen < 2022.8.23, we need to do it in a different `python` environment.

from doped.pycdt.core.defectsmaker import ChargedDefectsStructures
from pymatgen.core.structure import Structure

# Specify perfect (bulk) supercell structure
bulk_supercell = Structure.from_file("../tests/data/vasp/CdTe/CdTe_Bulk_Supercell_POSCAR")

# Let's generate the vacancies
cds = ChargedDefectsStructures(
    bulk_supercell,
    cellmax=bulk_supercell.num_sites,
    antisites_flag=False,  # Don't include antisites, as we only want to extract the Te interstitials here
)
doped_defect_dict = cds.defects

Setting up vacancies
Searching for Voronoi interstitial sites (this can take a little while)
Found the interstital sites, setting up interstitials
Defects generated:
Vacancies    Potential Charge States    Supercell Site Multiplicity
-----------  -------------------------  -----------------------------
vac_1_Cd     [-2, -1, 0, 1, 2]          32
vac_2_Te     [-2, -1, 0, 1, 2]          32 

Interstitials    Potential Charge States        Supercell Site Multiplicity
---------------  -----------------------------  -----------------------------
inter_1_Cd       [0, 1, 2]                      1
inter_2_Cd       [0, 1, 2]                      128
inter_3_Cd       [0, 1, 2]                      32
inter_4_Te       [-2, -1, 0, 1, 2, 3, 4, 5, 6]  32
inter_5_Te       [-2, -1, 0, 1, 2, 3, 4, 5, 6]  128
inter_6_Te       [-2, -1, 0, 1, 2, 3, 4, 5, 6]  1 



In [2]:
print("Interstitials generated:", [entry["name"] for entry in doped_defect_dict["interstitials"]])

Interstitials generated: ['inter_1_Cd', 'inter_2_Cd', 'inter_3_Cd', 'inter_4_Te', 'inter_5_Te', 'inter_6_Te']


We typically recommend to do Gamma-point relaxations for the neutral state of each generated interstitial candidate, and remove any that are high in energy (>1 eV above the lowest energy site) or that end up relaxing to the same final structure/energy as other interstitial sites.

For demonstration purposes, here we'll focus on `inter_6_Te`, which has been found to be the lowest energy Te interstitial.

In [3]:
# only using these species for example purpose:
Te_i_dict = {"interstitials": [inter for inter in doped_defect_dict["interstitials"] if inter["name"] == "inter_6_Te"],
             "bulk": doped_defect_dict["bulk"]} # We need the bulk entry for later
Te_i_dict["interstitials"][0]["charges"] = [0,-1,-2]  # include only the 0, -1 and -2 charge states

In [4]:
# save our Te_i dict to file, so we can reload and use with ShakeNBreak:
from monty.serialization import dumpfn
dumpfn(Te_i_dict, "../tests/data/vasp/CdTe/doped_Te_i_dict.json")

We now switch back to our `base`/`ShakeNBreak` environment and reload these generated defects from the `json` file, to use with `ShakeNBreak`:

In [2]:
# After generating the doped defects dict in a different environment and saving it to json,
# we load it in the ShakeNBreak environment:
from monty.serialization import loadfn

Te_i_dict = loadfn("../tests/data/vasp/CdTe/doped_Te_i_dict.json")  # path to saved defects json file

<a id='SnB'></a>

## 2. Apply the `SnB` method to your defects

The default settings and parameter choices in this package have been tested and have performed best thus far (i.e. wider distortion ranges leading to the ground-state structure with lowest computational cost) – see SI of [_Identifying the ground state structures of point defects in solids_](https://www.nature.com/articles/s41524-023-00973-1) Mosquera-Lois, Kavanagh, Walsh and Scanlon 2022.

If you encounter improved performance with non-default parameter choices, we'd love to know! Please get in touch via GitHub or by email: sean.kavanagh.19@ucl.ac.uk & i.mosquera-lois22@imperial.ac.uk

If you are investigating defects in hard/ionic/magnetic/correlated materials, or systems involving spectator ions (like A in ABX$_3$), there are some extra considerations for boosting the performance & efficiency of `SnB` listed on the [Miscellaneous Tips & Tricks](https://shakenbreak.readthedocs.io/en/latest/Tips.html) docs page.

### 2.1 Generating distorted structures

In [3]:
from shakenbreak import energy_lowering_distortions, input
from shakenbreak.input import Distortions

In [4]:
# In order to determine the number of the defect nearest neighbours to distort (based on the change 
# in valence electrons mentioned above), SnB uses the oxidation states of atoms in our material:
# If not specified, the code will guess these, otherwise you can specify as such:
# oxidation_states = {"Cd": +2, "Te": -2}  # specify atom oxidation states

# Create an instance of Distortion class with the defects and distortion parameters
# If distortion parameters are not specified, the default values are used
Dist = Distortions(defects=Te_i_dict)

Oxidation states were not explicitly set, thus have been guessed as {'Cd': 2.0, 'Te': -2.0}. If this is unreasonable you should manually set oxidation_states


The `Distortions()` class is flexible to the user input, so can take single `pymatgen` `Defect` objects, a list of `Defect`s, or a dictionary of `Defect`s (in which case the dictionary keys are used as the defect names) as inputs.

The defect dictionary output by `ChargedDefectStructures` in `doped` can also be used to initialise `Distortions`, as done in the previous cell above for Te interstitials.

These possibilities as well as the optional distortion parameters are detailed in the `Distortions` class docstring:

In [7]:
Distortions?

Init signature:
Distortions(
    defects: Union[list, dict],
    oxidation_states: Union[dict, NoneType] = None,
    padding: int = 1,
    dict_number_electrons_user: Union[dict, NoneType] = None,
    distortion_increment: float = 0.1,
    bond_distortions: Union[list, NoneType] = None,
    local_rattle: bool = False,
    distorted_elements: Union[dict, NoneType] = None,
    **mc_rattle_kwargs,
)
Docstring:     
Class to apply rattle and bond distortion to all defects in `defects`
(each defect as a pymatgen.analysis.defects.core.Defect() object).
Init docstring:
Args:
    defects (:obj:`dict_or_list_or_Defect`):
        List or dictionary of pymatgen.analysis.defects.core.Defect() objects.
        E.g.: [Vacancy(), Interstitial(), Substitution(), ...], or single Defect().
        In this case, generated defect folders will be named in the format:
        "{Defect.name}_m{Defect.multiplicity}" for interstitials and
        "{Defect.name}_s{Defect.defect_site_index}" for vacancies and su

In [5]:
# We can check the distortion parameters using some of the class properties
print(f"Bond distortions: {Dist.bond_distortions}")
print(f"Rattle standard deviation: {Dist.stdev:.2f} Å")  # set to 10% of the bulk bond length by default, typically a reasonable value

Bond distortions: [-0.6, -0.5, -0.4, -0.3, -0.2, -0.1, 0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6]
Rattle standard deviation: 0.28 Å


In [6]:
# You can restrict the ions that are distorted to a certain element using the keyword distorted_elements
# We can check it using the class attribute
print("User defined elements to distort:", Dist.distorted_elements)
# If None, it means no restrictions so nearest neighbours are distorted (recommended default, 
# unless you have reason to suspect otherwise; see shakenbreak.readthedocs.io/en/latest/Tips.html)

User defined elements to distort: None


If we're only interested in generating distorted structures, but not in writing `VASP`/other codes input files, we can use the class method `Distortions.apply_distortions()` to do this.

In [7]:
defects_dict, distortion_metadata = Dist.apply_distortions()

Applying ShakeNBreak... Will apply the following bond distortions: ['-0.6', '-0.5', '-0.4', '-0.3', '-0.2', '-0.1', '0.0', '0.1', '0.2', '0.3', '0.4', '0.5', '0.6']. Then, will rattle with a std dev of 0.28 Å 


Defect: inter_6_Te
Number of missing electrons in neutral state: 2

Defect inter_6_Te in charge state: 0. Number of distorted neighbours: 2

Defect inter_6_Te in charge state: -1. Number of distorted neighbours: 1

Defect inter_6_Te in charge state: -2. Number of distorted neighbours: 0


In [8]:
defects_dict["inter_6_Te"].keys()

dict_keys(['defect_type', 'defect_site', 'defect_supercell_site', 'defect_multiplicity', 'charges'])

In [10]:
# The output dictionary contains information about each defect:
print("Keys for each defect entry:", defects_dict["inter_6_Te"].keys())

# As well as the distorted structures for each charge state of all defects
# We can access the distorted structures of v_Cd_0 like this:
print("\nUndistorted and distorted structures:")
defects_dict["inter_6_Te"]["charges"][0]["structures"]

Keys for each defect entry: dict_keys(['defect_type', 'defect_site', 'defect_supercell_site', 'defect_multiplicity', 'charges'])

Undistorted and distorted structures:


{'Unperturbed': Structure Summary
 Lattice
     abc : 13.086768 13.086768 13.086768
  angles : 90.0 90.0 90.0
  volume : 2241.2856479961474
       A : 13.086768 0.0 0.0
       B : 0.0 13.086768 0.0
       C : 0.0 0.0 13.086768
     pbc : True True True
 PeriodicSite: Te4+ (11.4509, 1.6358, 11.4509) [0.8750, 0.1250, 0.8750]
 PeriodicSite: Cd2+ (0.0000, 0.0000, 0.0000) [0.0000, 0.0000, 0.0000]
 PeriodicSite: Cd2+ (0.0000, 0.0000, 6.5434) [0.0000, 0.0000, 0.5000]
 PeriodicSite: Cd2+ (0.0000, 6.5434, 0.0000) [0.0000, 0.5000, 0.0000]
 PeriodicSite: Cd2+ (0.0000, 6.5434, 6.5434) [0.0000, 0.5000, 0.5000]
 PeriodicSite: Cd2+ (6.5434, 0.0000, 0.0000) [0.5000, 0.0000, 0.0000]
 PeriodicSite: Cd2+ (6.5434, 0.0000, 6.5434) [0.5000, 0.0000, 0.5000]
 PeriodicSite: Cd2+ (6.5434, 6.5434, 0.0000) [0.5000, 0.5000, 0.0000]
 PeriodicSite: Cd2+ (6.5434, 6.5434, 6.5434) [0.5000, 0.5000, 0.5000]
 PeriodicSite: Cd2+ (0.0000, 3.2717, 3.2717) [0.0000, 0.2500, 0.2500]
 PeriodicSite: Cd2+ (0.0000, 3.2717, 9.8151) 

_(If you are viewing this on the [SnB Python API tutorial](https://shakenbreak.readthedocs.io/en/latest/ShakeNBreak_Example_Workflow.html) docs page, long output cells like this and printed dictionaries/structures below are scrollable!)_

### 2.2 Generating `VASP` input files for the distorted structures

If we want to generate `VASP` input files, we can use the class method `Distortions.write_vasp_files()` (instead of `Distortions.apply_distortions()`)

In [11]:
defects_dict, distortion_metadata = Dist.write_vasp_files()

Applying ShakeNBreak... Will apply the following bond distortions: ['-0.6', '-0.5', '-0.4', '-0.3', '-0.2', '-0.1', '0.0', '0.1', '0.2', '0.3', '0.4', '0.5', '0.6']. Then, will rattle with a std dev of 0.28 Å 


Defect: inter_6_Te
Number of missing electrons in neutral state: 2

Defect inter_6_Te in charge state: 0. Number of distorted neighbours: 2

Defect inter_6_Te in charge state: -1. Number of distorted neighbours: 1

Defect inter_6_Te in charge state: -2. Number of distorted neighbours: 0


Using the `incar_settings` optional argument for `Distortions.write_vasp_files()` above, we can also specify some custom `INCAR` tags to match our converged `ENCUT` for this system and optimal `NCORE` for the HPC we will run the calculations on. More information on the distortions generated can be obtained by setting `verbose = True`. Note that any `INCAR` flags that aren't numbers (e.g. `{"IBRION": 1}`) or True/False (e.g. `{"LREAL": False}`) need to be input as strings with quotation marks (e.g. `{"ALGO": "All"}`).

Our distorted structures and VASP input files have now been generated in the `Int_Te_3_X` folders.

For the recommended default coarse structure-searching `INCAR` settings, either have a look at the `incar.yaml` file in the `SnB_input_files` folder or at the generated files:

In [12]:
!cat ./inter_6_Te_0/Bond_Distortion_-10.0%/INCAR

# May want to change NCORE, KPAR, AEXX, ENCUT, NUPDOWN, ISPIN, POTIM = 
# ShakeNBreak INCAR with coarse settings to maximise speed with sufficient accuracy for qualitative structure searching = 
# KPAR = # No KPAR, only one kpoint
ALGO = Normal
EDIFFG = -0.01
ENCUT = 300
HFSCREEN = 0.2
IBRION = 2 # While often slower than '1' (RMM-DIIS), this is more stable and reliable, and vasp_gam relaxations are typically cheap enough to justify it
ISIF = 2
ISMEAR = 0
ISPIN = 2 # Spin polarisation likely for defects
ISYM = 0 # Symmetry breaking extremely likely for defects
LASPH = True
LCHARG = False
LHFCALC = True
LORBIT = 14
LREAL = Auto
LWAVE = False
NCORE = 12
NEDOS = 2000
NELM = 40
NSW = 300
PREC = Accurate
PRECFOCK = Fast
SIGMA = 0.05
NELECT = 582.0
NUPDOWN = 0 # But could be 2 if strong spin polarisation or magnetic behaviour present
EDIFF = 1e-05 # May need to reduce for tricky relaxations
ROPT = 1e-3 1e-3 1e-3


Note that the `NELECT` `INCAR` tag (number of electrons) is automatically determined based on the choice of `POTCAR`s. The default in `ShakeNBreak` is to use the [`MPRelaxSet` `POTCAR` choices](https://github.com/materialsproject/pymatgen/blob/master/pymatgen/io/vasp/MPRelaxSet.yaml), but if you're using different ones, make sure to set `potcar_settings` in `apply_shakenbreak()`, so that NELECT is then set accordingly. This requires the `pymatgen` config file `$HOME/.pmgrc.yaml` to be properly set up as detailed on the [GitHub `README`](https://github.com/SMTG-UCL/ShakeNBreak) and [Installation](https://shakenbreak.readthedocs.io/en/latest/Installation.html) docs page.

For generating the input files for other electronic structure codes (`Quantum Espresso`, `FHI-aims`, `CP2K`, `CASTEP`), see the [(Optional) Generate input files for other codes](#other) section at the end of this notebook.

<a id='HPCs'></a>

## 3. Send to HPCs and run relaxations

Can use the `snb-run` CLI function to quickly run calculations; see the [Submitting the geometry optimisations](https://shakenbreak.readthedocs.io/en/latest/Generation.html#submitting-the-geometry-optimisations) section of the CLI tutorial for this.

#### a) For `VASP` users:

Then parse the energies obtained by running the `snb-parse` command from the top-level folder containing your defect folders (e.g. `inter_6_Te_0` etc. (with subfolders: `inter_6_Te_0/Bond_Distortion_10.0%` etc.)). This will parse the energies and store them in a `inter_6_Te_0.yaml` etc file in the defect folders, to allow easy plotting and analysis.

It is also recommended to parse the final structures (`CONTCAR`s files if using `VASP`) obtained with each distortion relaxation for further structural analysis, which is done automatically when downloaded to your local folders as below. 

Copying these data to your local PC can be done quickly from your local folder top-level folder (containing `inter_6_Te_0` etc) with the following code:

```bash
for defect in ./*{_,_-}[0-9]/; do cd $defect; scp {remote_machine}:{path to ShakeNBreak folders}/${defect}${defect%?}.yaml .; for distortion in (Bond_Distortion|Unperturbed|Rattled)*/; do scp {remote_machine}:{path to ShakeNBreak folders}/${defect}${distortion}CONTCAR ${distortion}; done; cd ..; done
```
making sure to change `{remote_machine}` and `{path to ShakeNBreak folders}` to the correct values in your case.

#### b) If using `CP2K`, `Quantum Espresso`, `CASTEP` or `FHI-aims`:
Then parse the energies obtained by running the `snb-parse` command from the top-level folder containing your defect folders (e.g. `v_Cd_0` etc. (with subfolders: `v_Cd_0/Bond_Distortion_10.0%` etc.)) and setting the `--code` option (e.g. `snb-parse --code cp2k`). This will parse the energies and store them in a `v_Cd_0.yaml` etc file in the defect folders, to allow easy plotting and analysis. 

It is also recommended to parse the final structures obtained with each relaxation for further structural analysis. Depending on the code, the structure information is read from:
* `CP2K`: restart file
* `Quantum Espresso`: output file
* `CASTEP`: output file (i.e. `.castep`)
* `FHI-aims`: output file